<a href="https://colab.research.google.com/github/gyulgyulyan/MyScripts/blob/main/LTV_150_ad_em_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import os
import datetime as dt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
from scipy.misc import derivative
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import re
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
import lightgbm as lgb

from sqlalchemy import create_engine # !pip install clickhouse-sqlalchemy

In [ ]:
# выгрузка данных
df_merged=pd.read_csv('ml_em_jan_new.csv')

In [ ]:
quint=pd.read_csv('train_quantile.csv')
df_merged=pd.merge(df_merged, quint, on=["app_id", "country_installs"], how='left')

In [ ]:
df_merged=df_merged.loc[df_merged.device_id!='00000000-0000-0000-0000-000000000000']

In [ ]:
# получение временных фичей
df_merged['no_activity_last_3_days'] = df_merged.apply(lambda x: (1 if x['num_sessions_11'] * x['num_sessions_12'] * x['num_sessions_13'] > 0 else 0), axis = 1)
df_merged['install_date_utc'] = pd.to_datetime(df_merged['install_date_utc'])
df_merged['last_session_0_13'] = pd.to_datetime(df_merged['last_session_0_13'], format='%Y-%m-%d')
df_merged['max_date'] = df_merged['install_date_utc'] + dt.timedelta(14) 
df_merged['days_from_last_session'] = df_merged.apply(lambda x: x['max_date']-x['last_session_0_13'], axis=1)
df_merged['days_from_last_session'] = df_merged['days_from_last_session'].dt.days

In [ ]:
# фильтрация данных
df_merged=df_merged.loc[(df_merged.max_das<=13) | (df_merged.first_session>='2021-01-01')]
df_merged=df_merged.loc[(df_merged.days_from_last_session>=-1)]

df_merged['days_from_last_session'] = df_merged.apply(lambda x: 0 if x['days_from_last_session'] <0 else x['days_from_last_session'], axis = 1)
df_merged['days_from_last_session'] = df_merged.apply(lambda x: 13 if x['days_from_last_session']>13 else x['days_from_last_session'], axis = 1)

df_merged['daylife'] = df_merged.apply(lambda x: x['last_session_0_13']-x['install_date_utc'], axis=1)
df_merged['daylife'] = df_merged['daylife'].dt.days
df_merged['daylife']=np.where((df_merged.max_das<14) & ((df_merged.daylife<0) | (df_merged.daylife>13)),df_merged.max_das, 13-df_merged.days_from_last_session)

In [ ]:
df_merged['app_revenue_0_13']=df_merged['app_revenue_0']+df_merged['app_revenue_1']+ df_merged['app_revenue_2']+ df_merged['app_revenue_3']+ df_merged['app_revenue_4']+df_merged['app_revenue_5']+ df_merged['app_revenue_6']+ df_merged['app_revenue_7']+ df_merged['app_revenue_8']+df_merged['app_revenue_9']+ df_merged['app_revenue_10']+ df_merged['app_revenue_11']+ df_merged['app_revenue_12']+df_merged['app_revenue_13']

In [ ]:
features = ['uid_count', 'country_count', 'city_count', 'app_version_count', 
            'fs_visit_count', 'fs_visit_count_7_13', 'fs_visit_count_10_13', 
            'fs_visit_count_13', 'fs_visit_count_12', 'fs_visit_count_11',
            'is_signup', 'is_start_trial', 'ad_revenue_0','ad_revenue_1','ad_revenue_0_1',
       'ad_revenue_2', 'ad_revenue_3', 'ad_revenue_4', 'ad_revenue_5',
       'ad_revenue_6', 'ad_revenue_7', 'ad_revenue_8', 'ad_revenue_9',
       'ad_revenue_10', 'ad_revenue_11', 'ad_revenue_12', 'ad_revenue_13', 'ad_revenue_0_13', 
            'country_qol_idx', 'country_col_idx', 'is_capital',
            'app_revenue_0','app_revenue_0_1',
       'app_revenue_1', 'app_revenue_2', 'app_revenue_3', 'app_revenue_4',
       'app_revenue_5', 'app_revenue_6', 'app_revenue_7', 'app_revenue_8',
       'app_revenue_9', 'app_revenue_10', 'app_revenue_11', 'app_revenue_12',
       'app_revenue_13',
            'app_revenue_0_13',
            'das_count_0', 'das_count_1_3', 'das_count_4_7', 'das_count_7_13', 'das_count_0_13',
            'retention_0', 'retention_1', 'retention_2', 'retention_3', 'retention_4', 'retention_5',
            'retention_6', 'retention_7', 'retention_8', 'retention_9', 'retention_10', 'retention_11',
            'retention_12', 'retention_13',
            'num_sessions_0','num_sessions_1', 'num_sessions_2', 'num_sessions_3', 'num_sessions_4', 'num_sessions_5',
        'num_sessions_6', 'num_sessions_7', 'num_sessions_8', 'num_sessions_9', 'num_sessions_10', 'num_sessions_11',
        'num_sessions_12', 'num_sessions_13',
            'num_ads_0', 'num_ads_1', 'num_ads_2',
       'num_ads_3', 'num_ads_4', 'num_ads_5', 'num_ads_6', 'num_ads_7',
       'num_ads_8', 'num_ads_9', 'num_ads_10', 'num_ads_11', 'num_ads_12',
       'num_ads_13',
            
            'is_likely_bot', 'no_sessions', 
            'ads_per_session', 'income_per_ad',
            'num_sessions_bins', 'num_ads_bins',
            'num_dialogs_bins', 'photo_views_bins', 'dialogs_per_session', 'views_per_session',
            'has_install', 'niggers_group_0_13','vvp', 'daylife', 'sum_session_duration_0_13',
            'r_score','f_score','m_score','das_score','dial_score'
            ]

cat_features = ['country_signup_cat', 'media_source_cat', 'campaign_type_cat', 'top_age_group_', 
                'country_tier_cat', 'top_gender_cat', 'country_profile_cat', 'rfm']

#### Генерация всяких фичей

In [ ]:
df_merged['quan_20_das_count_0_13']=np.where(df_merged.quan_20_das_count_0_13>1,df_merged.quan_20_das_count_0_13,1)
df_merged['quan_20_num_dial_0_13']=np.where(df_merged.quan_20_num_dial_0_13>1,df_merged.quan_20_num_dial_0_13,1)

df_merged['r_score']=np.where(df_merged.days_from_last_session<=df_merged.quan_20_last_sess_0_13, 3,
                             np.where(df_merged.days_from_last_session<=df_merged.quan_80_last_sess_0_13, 2,1))
df_merged['f_score']=np.where(df_merged.num_sessions_0_13>df_merged.quan_80_sess_0_13, 3,
                             np.where(df_merged.num_sessions_0_13>1, 2,1))
df_merged['m_score']=np.where(df_merged.ad_revenue_0_13>df_merged.quan_80_ad_rev_0_13, 3,
                             np.where(df_merged.ad_revenue_0_13>0, 2,1))
df_merged['das_score']=np.where(df_merged.das_count_0_13>=df_merged.quan_80_das_count_0_13, 3,
                             np.where(df_merged.das_count_0_13>=df_merged.quan_20_das_count_0_13, 2,1))
df_merged['dial_score']=np.where(df_merged.days_from_last_session>=df_merged.quan_80_num_dial_0_13, 3,
                             np.where(df_merged.days_from_last_session>=df_merged.quan_20_num_dial_0_13, 2,1))

In [ ]:
df_merged['rfm'] = df_merged['r_score'].map(str) + df_merged['f_score'].map(str) + df_merged['m_score'].map(str) + df_merged['das_score'].map(str) + df_merged['dial_score'].map(str)
df_merged['rfm'] = df_merged['rfm'].astype('category')

In [ ]:
perc_sess = df_merged['num_sessions_0_13'].quantile([0.97]).values[0]
perc_ads = df_merged['num_ads_0_13'].quantile([0.97]).values[0]
df_merged['is_likely_bot'] = df_merged.apply(lambda x: (1 if (x['num_sessions_0_13'] >= perc_sess) or (x['num_ads_0_13'] >= perc_ads) else  0), axis = 1)
df_merged['no_sessions'] = df_merged.apply(lambda x: (1 if x['num_sessions_0_13'] == 0 else 0), axis = 1)

In [ ]:
df_merged['install_date_utc'] = pd.to_datetime(df_merged['install_date_utc'])
df_merged['ads_per_session'] = np.where(df_merged['num_sessions_0_13']==0,0,df_merged['num_ads_0_13'] / df_merged['num_sessions_0_13'])
df_merged['income_per_ad'] = np.where(df_merged['num_ads_0_13']==0,0,df_merged['ad_revenue_0_13'] / df_merged['num_ads_0_13'])
df_merged['dialogs_per_session'] = np.where(df_merged['num_sessions_0_13']==0,0,df_merged['num_dialogs_0_13'] / df_merged['num_sessions_0_13'])
df_merged['views_per_session'] = np.where(df_merged['num_sessions_0_13']==0,0,df_merged['photo_views_0_13'] / df_merged['num_sessions_0_13'])

def pct_rank_qcut(series, n):
    edges = pd.Series([float(i) / n for i in range(n + 1)])
    f = lambda x: (edges >= x).argmax()
    return series.rank(pct=1).apply(f)

df_merged['num_sessions_bins'] = pct_rank_qcut(df_merged['num_sessions_0_13'], 10)
df_merged['num_ads_bins'] = pct_rank_qcut(df_merged['num_ads_0_13'], 10)
df_merged['num_dialogs_bins'] = pct_rank_qcut(df_merged['num_dialogs_0_13'], 10)
df_merged['photo_views_bins'] = pct_rank_qcut(df_merged['photo_views_0_13'], 10)

In [ ]:
vvp=pd.read_csv('vvp.csv', sep=";", low_memory=False)
df_merged=pd.merge(df_merged,vvp, how='left')
df_merged.loc[pd.isna(df_merged.vvp)==True,'vvp']=int(vvp.loc[vvp.country_profile=='unknown','vvp'])

In [ ]:
df_merged[features] = df_merged[features].fillna(0)

#### Модель

In [ ]:
import datetime

class Ads150:
    def __init__(self,  features, cat_features):

        self._target = 'ad_revenue_14_150'

        self._model = LGBMRegressor(
            objective=self._custom_asymmetric_loss,
            metrics=['l2', 'l1'],
            n_estimators=200,
            random_state=20,
            max_depth=6, #8
            num_leaves = 40, #20
            verbose=0
        )
        

        self._init_model = None
        self._df = None
        
        self._features = features
        self._cat_features = cat_features 

    def pre_fit(self):
        pass

    def _custom_asymmetric_loss(self, y_true, y_pred):
        residual = (y_true - y_pred).astype('float')
        grad = np.where(residual > 0, -2 * 1.0 * residual, -2 * 1.0 * residual) 
        hess = np.where(residual > 0, 2 * 1.0, 2.0 * 1.0)
        return grad, hess

    def predict(self, dataset):
        dataset['predict_origin'] = self._model.predict(dataset[self._features+self._cat_features])
        dataset['predict'] = dataset.apply(lambda x: max(x['predict_origin'], 0), axis=1)
        dataset['count'] = dataset.apply(lambda x: 1, axis=1)
        
        dataset['predict'] = dataset['predict'] + dataset['ad_revenue_0_13']
        
        return dataset
    
    def fit(self, df,k):
        self._df = df
        
#         df[self._cat_features] = df[self._cat_features].astype('category')

        if k==0:
            self._model.fit(
                    X=self._df[self._features+self._cat_features],
                    y=self._df[self._target]
            )

        else:
            self._model.fit(
                    X=self._df[self._features+self._cat_features],
                    y=self._df[self._target],
                    init_model=self._model
            )
        self._model.booster_.save_model('model_adrev_em.txt')

        return self._model.feature_importances_
 

In [ ]:
m = Ads150(features, cat_features)

dff_train = {n: df_merged.iloc[n:n+500000, :] 
           for n in range(0, len(df_merged), 500000)}
for n in dff_train.keys():
    if n==0:
        importance = m.fit(dff_train[n],0)
    else:
        importance = m.fit(dff_train[n],1)
    

[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.071844 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f